In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#abrir formulario


driver.get('https://sivireno.undc.edu.pe/index_login.php')
driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/form/div[1]/a').click()
driver.find_element(by=By.XPATH, value='//*[@id="identifierId"]').send_keys(CREDENTIAL["EMAIL"])
driver.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button/span').click()
time.sleep(2)
driver.find_element(by=By.XPATH, value='//*[@id="password"]/div[1]/div/div[1]/input').send_keys(CREDENTIAL["PASSWORD"])
driver.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button/span').click()
time.sleep(2)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:04<00:00, 1.77MB/s]


In [59]:
url="https://sivireno.undc.edu.pe/index_home.php?s=asistencia_nolectiva.php&id_doce=378"
driver.get(url)

In [2]:
#SCRIPT CARGA NO LECTIVA DE MANERA MASIVA
import pandas as pd
import datetime
df = pd.read_excel('EXCEL_TRABAJADO/2023-1cnl.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]',
               '17':'//*[@id="exampleFormControlSelect1"]/option[17]',
               '18':'//*[@id="exampleFormControlSelect1"]/option[18]',
               '19':'//*[@id="exampleFormControlSelect1"]/option[19]',
               '20':'//*[@id="exampleFormControlSelect1"]/option[20]',
               '21':'//*[@id="exampleFormControlSelect1"]/option[21]',
               '22':'//*[@id="exampleFormControlSelect1"]/option[22]',
               '23':'//*[@id="exampleFormControlSelect1"]/option[23]',
               '24':'//*[@id="exampleFormControlSelect1"]/option[24]',
               '25':'//*[@id="exampleFormControlSelect1"]/option[25]',
               '26':'//*[@id="exampleFormControlSelect1"]/option[26]'}

#INICIO DE CLASES
w_iclass={'LUNES':'27/03/2023',
               'MARTES':'28/03/2023',
               'MIERCOLES':'29/03/2023',
               'JUEVES':'30/03/2023',
               'VIERNES':'31/03/2023',
               'SABADO':'01/04/2023',
               'DOMINGO':'02/04/2023'}

urltemp=""
############################################            METODOS
    #ALGORITMO PARA LA FECHA
def convertDateWrite(fecha):
    f = str(fecha).split('-')
    año = f[0]
    mes = f[1]
    d= f[2].split()
    dia = d[0]
    return dia+mes+año
    
    #ALGORTIMO HORA
def convertHourWrite(hour):
    h = str(hour).split(':')
    return h[0]+h[1]

    # 7 + W, CON RESPECTO AL DIA
def sumWeek(dia,sem):
    return convertDateWrite(datetime.datetime.strptime(w_iclass[dia], '%d/%m/%Y')+(datetime.timedelta(days=7)*(sem-1)))

def writeForm(semi,semf,dia,doc,horae,horas):
    for j in range(int(semi),int(semf+1)):
        print("Docente :"+doc+" j:"+str(j)+" semi:"+str(semi)+" semf:"+str(semf)+" fecha i:"+sumWeek(dia,j))
        time.sleep(0.140)
        #ingresar semana
        driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(j))]).click()
        time.sleep(0.150)
        #ingresar fecha
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(sumWeek(dia,j))

        #ingresar fecha entrada
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(convertHourWrite(horae)))
        #ingresar fecha salida
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(convertHourWrite(horas)))

        #enviar registro
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[9]/button').click()
        time.sleep(0.150)
############################################            METODOS/*

for row, datos in df.iterrows():
    if datos['FLAG'] == 'SI':
        url = datos['URL']
        semi = datos['SEM_I']
        semf = datos['SEM_F']
        dia = datos['DIA']
        doc=datos['DOCENTE']
        horae = datos['HORAE']
        horas = datos['HORAS']


        if urltemp==url:
            writeForm(semi,semf,dia,doc,horae,horas)
        else:
            if str(semi)!="nan":   
                urltemp=url
                time.sleep(2)
                driver.get(url)
                time.sleep(1.5)
                writeForm(semi,semf,dia,doc,horae,horas)
            

Docente :DAMIAN SALAZAR ROJAS j:1 semi:1.0 semf:16.0 fecha i:29032023
Docente :DAMIAN SALAZAR ROJAS j:2 semi:1.0 semf:16.0 fecha i:05042023
Docente :DAMIAN SALAZAR ROJAS j:3 semi:1.0 semf:16.0 fecha i:12042023
Docente :DAMIAN SALAZAR ROJAS j:4 semi:1.0 semf:16.0 fecha i:19042023
Docente :DAMIAN SALAZAR ROJAS j:5 semi:1.0 semf:16.0 fecha i:26042023
Docente :DAMIAN SALAZAR ROJAS j:6 semi:1.0 semf:16.0 fecha i:03052023
Docente :DAMIAN SALAZAR ROJAS j:7 semi:1.0 semf:16.0 fecha i:10052023
Docente :DAMIAN SALAZAR ROJAS j:8 semi:1.0 semf:16.0 fecha i:17052023
Docente :DAMIAN SALAZAR ROJAS j:9 semi:1.0 semf:16.0 fecha i:24052023
Docente :DAMIAN SALAZAR ROJAS j:10 semi:1.0 semf:16.0 fecha i:31052023
Docente :DAMIAN SALAZAR ROJAS j:11 semi:1.0 semf:16.0 fecha i:07062023
Docente :DAMIAN SALAZAR ROJAS j:12 semi:1.0 semf:16.0 fecha i:14062023
Docente :DAMIAN SALAZAR ROJAS j:13 semi:1.0 semf:16.0 fecha i:21062023
Docente :DAMIAN SALAZAR ROJAS j:14 semi:1.0 semf:16.0 fecha i:28062023
Docente :DAMIAN

In [57]:
#SCRIPT POR INDIVIDUAL EXCEL

import pandas as pd
df = pd.read_excel('carganolectiva.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]',
               '17':'//*[@id="exampleFormControlSelect1"]/option[17]',
               '18':'//*[@id="exampleFormControlSelect1"]/option[18]',
               '19':'//*[@id="exampleFormControlSelect1"]/option[19]',
               '20':'//*[@id="exampleFormControlSelect1"]/option[20]',
               '21':'//*[@id="exampleFormControlSelect1"]/option[21]',
               '22':'//*[@id="exampleFormControlSelect1"]/option[22]',
               '23':'//*[@id="exampleFormControlSelect1"]/option[23]',
               '24':'//*[@id="exampleFormControlSelect1"]/option[24]',
               '25':'//*[@id="exampleFormControlSelect1"]/option[25]',
               '26':'//*[@id="exampleFormControlSelect1"]/option[26]'}

for row, datos in df.iterrows():
    semana = datos['SEMANA']
    fecha = datos['FECHA']
    horae = datos['HORAE']
    horas = datos['HORAS']
    
    #ALGORITMO PARA LA FECHA
    f = str(fecha).split('-')
    año = f[0]
    mes = f[1]
    d= f[2].split()
    dia = d[0]
    fecha_resolve=[dia+mes+año]
    
    #ALGORTIMO HORA
    h1 = str(horae).split(':')
    horae=[h1[0]+h1[1]]
    h2 = str(horas).split(':')
    horas=[h2[0]+h2[1]]
    

    
    #ingresar semana
    driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(semana))]).click()
     
    #ingresar fecha
    driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(fecha_resolve)
    
    #ingresar fecha entrada
    driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(horae))

    #ingresar fecha salida
    driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(horas))
    
    #enviar registro
    #driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[9]/button').click()
    
    

2022-10-01 15:42:42.168257
2022-09-29 15:42:42.168257
30/9/2022
08092022
ssss
08092022
j:1 semi:1 semf:3
j:2 semi:2 semf:3
j:3 semi:3 semf:3
